# Produce input catalogs for lensed system postage stamp generation

_Last Updated: Bryce Kalmbach, December 2018_

This is a notebook to produce the host AGN and SN catalogs that are used to created FITS postage stamps of the lensed systems. The goal here is to associate the lens and host information into a single catalog so that Nan Li can use it properly model the host galaxy shapes and we include them as fits files with the instance catalogs. PhoSim is able to take these fits files as input for the given line in the instance catalog so that we have properly shaped host galaxies in our lensed systems.

## Load in galaxy catalogs as dataframes

These come from unsprinkled instance catalogs of the DDF region.

In [1]:
import pandas as pd
from astropy.io import fits
import numpy as np
from desc.sims.GCRCatSimInterface import InstanceCatalogWriter
from lsst.sims.utils import SpecMap
import matplotlib.pyplot as plt
from lsst.utils import getPackageDir
from lsst.sims.photUtils import Sed, BandpassDict, Bandpass
from lsst.sims.catUtils.matchSED import matchBase
import os
%matplotlib inline

/global/common/software/lsst/common/miniconda/py3-4.3.21-env/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/global/homes/b/brycek/DC2/sims_catalogs/python/lsst/sims/catalogs/db/dbConnection.py:555: UserWarning: Duplicate object type id 4 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/global/homes/b/brycek/DC2/sims_catalogs/python/lsst/sims/catalogs/db/dbConnection.py:555: UserWarning: Duplicate object type id 28 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [2]:
base_columns = ['prefix', 'uniqueId', 'raPhoSim', 'decPhoSim',
                        'phosimMagNorm', 'sedFilepath', 'redshift',
                        'shear1', 'shear2', 'kappa', 'raOffset', 'decOffset',
                        'spatialmodel']

In [3]:
df_galaxy = pd.read_csv(os.path.join(os.environ['SCRATCH'],
                                     'bulge_gal_cat_197356.txt.gz'),
                                delimiter=' ', header=None,
                                names=base_columns+['majorAxis', 'minorAxis',
                                                    'positionAngle', 'sindex',
                                                    'internalExtinctionModel',
                                                    'internalAv', 'internalRv',
                                                    'galacticExtinctionModel',
                                                    'galacticAv', 'galacticRv'])

In [4]:
df_disk = pd.read_csv(os.path.join(os.environ['SCRATCH'],
                                     'disk_gal_cat_197356.txt.gz'),
                                delimiter=' ', header=None,
                                names=base_columns+['majorAxis', 'minorAxis',
                                                    'positionAngle', 'sindex',
                                                    'internalExtinctionModel',
                                                    'internalAv', 'internalRv',
                                                    'galacticExtinctionModel',
                                                    'galacticAv', 'galacticRv'])

In [5]:
df_agn = pd.read_csv(os.path.join(os.environ['SCRATCH'],
                                     'agn_gal_cat_197356.txt.gz'),
                                 delimiter=' ', header=None,
                                 names=base_columns+['internalExtinctionModel',
                                                     'galacticExtinctionModel',
                                                     'galacticAv', 'galacticRv'])

In [6]:
df_agn['galaxy_id'] = np.right_shift(df_agn['uniqueId'], 10)

In [7]:
df_agn.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,raOffset,decOffset,spatialmodel,internalExtinctionModel,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id
0,object,3392000069749,54.351016,-29.474949,21.245477,agnSED/agn.spec.gz,0.047309,-0.000002,-0.000048,0.000007,0,0,point,none,CCM,0.029307,3.1,3312500068
1,object,3392000201845,55.064013,-29.107730,19.310657,agnSED/agn.spec.gz,0.027914,-0.000015,0.000021,-0.000026,0,0,point,none,CCM,0.045022,3.1,3312500197
2,object,3392000248949,54.820543,-29.341426,20.232832,agnSED/agn.spec.gz,0.044629,-0.000012,0.000021,-0.000017,0,0,point,none,CCM,0.036800,3.1,3312500243
3,object,3392000249973,54.847392,-29.342723,22.711512,agnSED/agn.spec.gz,0.044629,-0.000008,0.000030,-0.000014,0,0,point,none,CCM,0.038021,3.1,3312500244
4,object,3392000252021,54.772554,-29.279037,23.265985,agnSED/agn.spec.gz,0.044629,-0.000055,0.000038,0.000025,0,0,point,none,CCM,0.035074,3.1,3312500246


In [8]:
df_galaxy['galaxy_id'] = np.right_shift(df_galaxy['uniqueId'], 10)

In [9]:
df_galaxy.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,minorAxis,positionAngle,sindex,internalExtinctionModel,internalAv,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id
0,object,3392000025697,54.881856,-28.765553,21.346167,galaxySED/Burst.80E09.002Z.spec.gz,0.017818,-0.000000,-0.000000,0.000000,...,0.648529,80.872978,4,CCM,0.0,2.0,CCM,0.044331,3.1,3312500025
1,object,3392000033889,54.815604,-28.803041,22.967427,galaxySED/Exp.80E09.002Z.spec.gz,0.021390,-0.000000,-0.000000,0.000000,...,0.298480,88.921657,4,CCM,0.0,2.0,CCM,0.044144,3.1,3312500033
2,object,3392000037985,55.234418,-29.034768,23.699069,galaxySED/Exp.10E10.002Z.spec.gz,0.018474,-0.000000,-0.000000,0.000000,...,1.166655,129.902200,4,CCM,0.0,2.0,CCM,0.047380,3.1,3312500037
3,object,3392000069729,54.351016,-29.474949,15.274427,galaxySED/Exp.10E10.04Z.spec.gz,0.047309,-0.000002,-0.000048,0.000007,...,2.746652,58.996228,4,CCM,0.0,2.0,CCM,0.029307,3.1,3312500068
4,object,3392000070753,54.913149,-29.205063,21.510131,galaxySED/Burst.80E09.04Z.spec.gz,0.037030,-0.000018,0.000027,-0.000017,...,0.708551,129.172123,4,CCM,0.0,2.0,CCM,0.039536,3.1,3312500069


In [10]:
df_disk['galaxy_id'] = np.right_shift(df_disk['uniqueId'], 10)

In [11]:
df_disk.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,minorAxis,positionAngle,sindex,internalExtinctionModel,internalAv,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id
0,object,3392000009323,54.693652,-29.077189,18.192925,galaxySED/Exp.20E09.02Z.spec.gz,0.020022,-0.0,-0.0,0.0,...,2.121733,3.872094,1,CCM,0.1,4.0,CCM,0.041432,3.1,3312500009
1,object,3392000016491,54.877503,-28.839979,20.971618,galaxySED/Burst.20E09.0005Z.spec.gz,0.018648,-0.0,-0.0,0.0,...,2.155571,79.758104,1,CCM,0.1,3.5,CCM,0.046292,3.1,3312500016
2,object,3392000019563,54.994210,-28.802359,22.405348,galaxySED/Exp.50E09.0005Z.spec.gz,0.017085,-0.0,-0.0,0.0,...,0.557445,157.899908,1,CCM,0.2,2.1,CCM,0.045183,3.1,3312500019
3,object,3392000022635,54.562747,-29.058309,23.121197,galaxySED/Exp.50E09.002Z.spec.gz,0.020561,-0.0,-0.0,0.0,...,1.082256,93.739670,1,CCM,0.1,2.8,CCM,0.034288,3.1,3312500022
4,object,3392000025707,54.881856,-28.765553,23.794633,galaxySED/Exp.62E09.0005Z.spec.gz,0.017818,-0.0,-0.0,0.0,...,0.794547,80.872978,1,CCM,0.1,2.9,CCM,0.044331,3.1,3312500025


## Make the AGN catalogs

What we are going to do is take the information on the unlensed bulge and disks from the instance catalog above and combine it with information on the lens that will be sprinkled in front of it so that we can accurately model the appearance of the lensed hosts in a fits postage stamp to include with the instance catalogs and give us properly modeled host galaxy images. We first do it for the AGN here and then repeat for the SNe below.

### Load in the AGN lensed systems and cache file

The cache file associates the `galaxy_id` values with the `twinklesId` values in the lensed system catalog. The file `cosmoDC2_vX.X_matched_AGN.fits` contains the information on the lensed systems.

In [12]:
catalog_version = 'cosmoDC2_v1.1.4'

In [13]:
hdulist = fits.open('../../data/%s_matched_AGN.fits' % catalog_version)
twinkles_lenses = hdulist[1].data

In [14]:
agn_cache_df = pd.read_csv('../../data/%s_agn_cache.csv' % catalog_version)

Next we add in the lens properties to go with each of the images.

In [15]:
#TwinklesId can range from 0 - 1099
ra_images = []
dec_images = []
num_images = []
twinkles_system_images = []
unique_id_lens = []

unique_id_bulge = []
unique_id_disk = []

bulge_index = []
disk_index = []

disk_valid = []

i = 0
for twinkles_system in range(len(twinkles_lenses)):
    
    if twinkles_system % 100 == 0:
        print(twinkles_system)
    
    cache_info = agn_cache_df.query('twinkles_system == %i' % twinkles_system)
    gal_id = cache_info['galtileid'].values[0]
    lens_info = twinkles_lenses[np.where(twinkles_lenses['twinklesId'] == twinkles_system)]
    bulge_info = df_galaxy.query('galaxy_id == %i' % gal_id)
    disk_info = df_disk.query('galaxy_id == %i' % gal_id)

    
    #print(bulge_info, disk_info)
    
    for lens_image in range(lens_info[0]['NIMG']):
        ra_shift = np.radians(lens_info[0]['XIMG'][lens_image] / 3600.) / np.cos(np.radians(bulge_info['decPhoSim']))
        dec_shift = np.radians(lens_info[0]['YIMG'][lens_image] / 3600.)
        
        image_ra = np.radians(bulge_info['raPhoSim']) + ra_shift
        image_dec = np.radians(bulge_info['decPhoSim'] + dec_shift)
        
        ra_images.append(np.degrees(image_ra).values[0])
        dec_images.append(np.degrees(image_dec).values[0])
        num_images.append(lens_image)
        unique_id_lens.append(bulge_info['uniqueId'].values[0])
        twinkles_system_images.append(twinkles_system)
        base_id = (gal_id + int(1.5e10))*10000 + twinkles_system*4 + lens_image
        unique_id_bulge.append(np.left_shift(base_id, 10) + 97)
        unique_id_disk.append(np.left_shift(base_id, 10) + 107)
        
        bulge_index.append(bulge_info.index[0])
        
        if len(disk_info) == 0:
            print(twinkles_system)
        else:
            disk_index.append(disk_info.index[0])
            disk_valid.append(i)
            
        i += 1

0
100
200
300
400
500
600
700
800
900
1000


In [16]:
host_bulge = df_galaxy.iloc[bulge_index]

In [17]:
host_bulge = host_bulge.reset_index(drop=True)

In [18]:
host_bulge['twinkles_system'] = twinkles_system_images

In [19]:
host_bulge['image_number'] = num_images

In [20]:
host_bulge['uniqueId_lens'] = unique_id_lens

In [21]:
host_bulge['uniqueId'] = unique_id_bulge

In [22]:
host_bulge['raPhoSim_lens'] = host_bulge['raPhoSim']
host_bulge['decPhoSim_lens'] = host_bulge['decPhoSim']

In [23]:
host_bulge['raPhoSim'] = ra_images
host_bulge['decPhoSim'] = dec_images

In [24]:
host_bulge.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id,twinkles_system,image_number,uniqueId_lens,raPhoSim_lens,decPhoSim_lens
0,object,179979750236160097,53.205292,-28.167426,22.932481,galaxySED/Exp.25E09.02Z.spec.gz,2.110945,0.009374,0.009484,-0.020661,...,3.0,CCM,0.021773,3.1,2576147484,0,0,2637975023713,53.205275,-28.167419
1,object,179979750236161121,53.205254,-28.167417,22.932481,galaxySED/Exp.25E09.02Z.spec.gz,2.110945,0.009374,0.009484,-0.020661,...,3.0,CCM,0.021773,3.1,2576147484,0,1,2637975023713,53.205275,-28.167419
2,object,179968158023684193,52.671304,-28.400567,23.592174,galaxySED/Exp.12E09.1Z.spec.gz,2.014421,0.010362,-0.014592,-0.050510,...,2.0,CCM,0.027994,3.1,2575015432,1,0,2636815802465,52.671334,-28.400563
3,object,179968158023685217,52.671311,-28.400551,23.592174,galaxySED/Exp.12E09.1Z.spec.gz,2.014421,0.010362,-0.014592,-0.050510,...,2.0,CCM,0.027994,3.1,2575015432,1,1,2636815802465,52.671334,-28.400563
4,object,179926356316168289,53.649285,-28.200846,23.750343,galaxySED/Exp.12E09.1Z.spec.gz,1.907144,0.011577,-0.035357,-0.027762,...,2.0,CCM,0.025911,3.1,2570933234,2,0,2632635631713,53.649314,-28.200843


In [25]:
host_disk = df_disk.iloc[disk_index]

In [26]:
host_disk = host_disk.reset_index(drop=True)

In [27]:
host_disk['twinkles_system'] = np.array(twinkles_system_images)[disk_valid]

In [28]:
host_disk['image_number'] = np.array(num_images)[disk_valid]

In [29]:
host_disk['uniqueId_lens'] = np.array(unique_id_lens)[disk_valid]

In [30]:
host_disk['uniqueId'] = np.array(unique_id_disk)[disk_valid]

In [31]:
host_disk['raPhoSim_lens'] = host_disk['raPhoSim']
host_disk['decPhoSim_lens'] = host_disk['decPhoSim']

In [32]:
host_disk['raPhoSim'] = np.array(ra_images)[disk_valid]
host_disk['decPhoSim'] = np.array(dec_images)[disk_valid]

In [33]:
host_disk.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id,twinkles_system,image_number,uniqueId_lens,raPhoSim_lens,decPhoSim_lens
0,object,179979750236160107,53.205292,-28.167426,52.535020,galaxySED/Inst.25E07.002Z.spec.gz,2.110945,0.009374,0.009484,-0.020661,...,2.0,CCM,0.021773,3.1,2576147484,0,0,2637975023713,53.205275,-28.167419
1,object,179979750236161131,53.205254,-28.167417,52.535020,galaxySED/Inst.25E07.002Z.spec.gz,2.110945,0.009374,0.009484,-0.020661,...,2.0,CCM,0.021773,3.1,2576147484,0,1,2637975023713,53.205275,-28.167419
2,object,179968158023684203,52.671304,-28.400567,48.186422,galaxySED/Const.64E07.002Z.spec.gz,2.014421,0.010362,-0.014592,-0.050510,...,2.0,CCM,0.027994,3.1,2575015432,1,0,2636815802465,52.671334,-28.400563
3,object,179968158023685227,52.671311,-28.400551,48.186422,galaxySED/Const.64E07.002Z.spec.gz,2.014421,0.010362,-0.014592,-0.050510,...,2.0,CCM,0.027994,3.1,2575015432,1,1,2636815802465,52.671334,-28.400563
4,object,179926356316168299,53.649285,-28.200846,49.271284,galaxySED/Inst.19E07.002Z.spec.gz,1.907144,0.011577,-0.035357,-0.027762,...,2.0,CCM,0.025911,3.1,2570933234,2,0,2632635631713,53.649314,-28.200843


In [34]:
host_bulge.to_csv('../../data/%s_bulge_agn_host.csv' % catalog_version)
host_disk.to_csv('../../data/%s_disk_agn_host.csv' % catalog_version)

## Make the SNe catalogs

We do the same as we did for the lensed AGN systems with the lensed SNe systems below.

### Load in the SNE lensed systems and cache file

The cache file associates the `galaxy_id` values with the `twinklesId` values in the lensed system catalog. The file `cosmoDC2_vX.X_sne_cat.csv` contains the information on the images.

In [35]:
sne_df = pd.read_csv('../../data/%s_sne_cat.csv' % catalog_version)

In [36]:
sne_cache_df = pd.read_csv('../../data/%s_sne_cache.csv' % catalog_version)

In [37]:
sne_df.head()

,t0,sigma,gamma,e,theta_e,theta_gamma,zs,zl,snx,sny,...,lensgal_sed,lens_av,lens_rv,twinkles_sysno,td,x,y,mu,imno,t_start
0,62789.969968,220.313202,0.014794,0.184035,262.226643,273.367425,0.092126,0.045038,0.528398,-0.214797,...,galaxySED/Exp.10E10.04Z.spec.gz,0.0,2.0,1100,0.000000,1.112007,-0.562962,2.611519,0,62789.969968
1,62789.969968,220.313202,0.014794,0.184035,262.226643,273.367425,0.092126,0.045038,0.528398,-0.214797,...,galaxySED/Exp.10E10.04Z.spec.gz,0.0,2.0,1100,8.476256,-0.106035,0.038406,-0.168308,1,62798.446224
2,61307.102728,219.712380,0.099394,0.468707,311.306044,228.228920,0.095760,0.043930,-0.237093,0.146633,...,galaxySED/Exp.40E09.1Z.spec.gz,0.1,4.0,1101,0.000000,-0.761709,0.609792,2.692527,0,61307.102728
3,61307.102728,219.712380,0.099394,0.468707,311.306044,228.228920,0.095760,0.043930,-0.237093,0.146633,...,galaxySED/Exp.40E09.1Z.spec.gz,0.1,4.0,1101,4.117691,0.456249,-0.093761,-2.305423,1,61311.220420
4,61018.143316,182.689346,0.066827,0.721095,35.788858,216.450284,0.151658,0.068575,-0.314648,0.041791,...,galaxySED/Exp.10E10.04Z.spec.gz,0.0,2.0,1102,0.000000,-0.838950,-0.247964,1.465403,0,61018.143316


Next we add in the lens properties to go with each of the images.

In [38]:
#TwinklesId ranges from 1100 - 1100+number of cached SNe
ra_images = []
dec_images = []
num_images = []
twinkles_system_images = []
unique_id_lens = []

unique_id_bulge = []
unique_id_disk = []

bulge_index = []
disk_index = []

disk_valid = []

i = 0
for twinkles_system in range(1100, 1100+len(sne_cache_df)):
    
    if twinkles_system % 100 == 0:
        print(twinkles_system)
    
    cache_info = sne_cache_df.query('twinkles_system == %i' % twinkles_system)
    gal_id = cache_info['galtileid'].values[0]
    lens_info = sne_df.iloc[np.where(sne_df['twinkles_sysno'] == twinkles_system)]
    bulge_info = df_galaxy.query('galaxy_id == %i' % gal_id)
    disk_info = df_disk.query('galaxy_id == %i' % gal_id)

    
    #print(bulge_info, disk_info)
    
    for lens_image in lens_info['imno'].values:
        ra_shift = np.radians(lens_info['x'].iloc[lens_image] / 3600.) / np.cos(np.radians(bulge_info['decPhoSim']))
        dec_shift = np.radians(lens_info['y'].iloc[lens_image] / 3600.)
        
        image_ra = np.radians(bulge_info['raPhoSim']) + ra_shift
        image_dec = np.radians(bulge_info['decPhoSim'] + dec_shift)
        
        ra_images.append(np.degrees(image_ra).values[0])
        dec_images.append(np.degrees(image_dec).values[0])
        num_images.append(lens_image)
        unique_id_lens.append(bulge_info['uniqueId'].values[0])
        twinkles_system_images.append(twinkles_system)
        base_id = (gal_id + int(1.5e10))*10000 + twinkles_system*4 + lens_image
        unique_id_bulge.append(np.left_shift(base_id, 10) + 97)
        unique_id_disk.append(np.left_shift(base_id, 10) + 107)
        
        bulge_index.append(bulge_info.index[0])
        
        if len(disk_info) == 0:
            print(twinkles_system)
        else:
            disk_index.append(disk_info.index[0])
            disk_valid.append(i)
            
        i += 1

1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100


In [39]:
host_bulge = df_galaxy.iloc[bulge_index]

In [40]:
host_bulge = host_bulge.reset_index(drop=True)

In [41]:
host_bulge['twinkles_system'] = twinkles_system_images

In [42]:
host_bulge['image_number'] = num_images

In [43]:
host_bulge['uniqueId_lens'] = unique_id_lens

In [44]:
host_bulge['uniqueId'] = unique_id_bulge

In [45]:
host_bulge['raPhoSim_lens'] = host_bulge['raPhoSim']
host_bulge['decPhoSim_lens'] = host_bulge['decPhoSim']

In [46]:
host_bulge['raPhoSim'] = ra_images
host_bulge['decPhoSim'] = dec_images

In [47]:
host_bulge.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id,twinkles_system,image_number,uniqueId_lens,raPhoSim_lens,decPhoSim_lens
0,object,179840022348185697,53.131556,-28.599572,21.059599,galaxySED/Exp.62E09.04Z.spec.gz,0.090045,0.000021,0.001851,-0.000146,...,4.0,CCM,0.029268,3.1,2562502182,1100,0,2624002234465,53.131204,-28.599570
1,object,179840022348186721,53.131170,-28.599570,21.059599,galaxySED/Exp.62E09.04Z.spec.gz,0.090045,0.000021,0.001851,-0.000146,...,4.0,CCM,0.029268,3.1,2562502182,1100,1,2624002234465,53.131204,-28.599570
2,object,179840029075869793,53.605522,-28.387494,25.743250,galaxySED/Burst.62E09.002Z.spec.gz,0.095564,0.000686,0.000152,-0.000170,...,4.0,CCM,0.027995,3.1,2562502839,1101,0,2624002907233,53.605763,-28.387497
3,object,179840029075870817,53.605907,-28.387497,25.743250,galaxySED/Burst.62E09.002Z.spec.gz,0.095564,0.000686,0.000152,-0.000170,...,4.0,CCM,0.027995,3.1,2562502839,1101,1,2624002907233,53.605763,-28.387497
4,object,179840093598113889,53.231066,-28.457710,25.455733,galaxySED/Inst.50E09.02Z.spec.gz,0.153528,0.000830,0.001565,-0.000518,...,4.0,CCM,0.025146,3.1,2562509140,1102,0,2624009359457,53.231331,-28.457709


In [48]:
host_disk = df_disk.iloc[disk_index]

In [49]:
host_disk = host_disk.reset_index(drop=True)

In [50]:
host_disk['twinkles_system'] = np.array(twinkles_system_images)[disk_valid]

In [51]:
host_disk['image_number'] = np.array(num_images)[disk_valid]

In [52]:
host_disk['uniqueId_lens'] = np.array(unique_id_lens)[disk_valid]

In [53]:
host_disk['uniqueId'] = np.array(unique_id_disk)[disk_valid]

In [54]:
host_disk['raPhoSim_lens'] = host_disk['raPhoSim']
host_disk['decPhoSim_lens'] = host_disk['decPhoSim']

In [55]:
host_disk['raPhoSim'] = np.array(ra_images)[disk_valid]
host_disk['decPhoSim'] = np.array(dec_images)[disk_valid]

In [56]:
host_disk.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id,twinkles_system,image_number,uniqueId_lens,raPhoSim_lens,decPhoSim_lens
0,object,179840022348185707,53.131556,-28.599572,20.286594,galaxySED/Const.12E10.04Z.spec.gz,0.090045,0.000021,0.001851,-0.000146,...,4.0,CCM,0.029268,3.1,2562502182,1100,0,2624002234465,53.131204,-28.599570
1,object,179840022348186731,53.131170,-28.599570,20.286594,galaxySED/Const.12E10.04Z.spec.gz,0.090045,0.000021,0.001851,-0.000146,...,4.0,CCM,0.029268,3.1,2562502182,1100,1,2624002234465,53.131204,-28.599570
2,object,179840029075869803,53.605522,-28.387494,25.172322,galaxySED/Const.10E10.0005Z.spec.gz,0.095564,0.000686,0.000152,-0.000170,...,2.3,CCM,0.027995,3.1,2562502839,1101,0,2624002907233,53.605763,-28.387497
3,object,179840029075870827,53.605907,-28.387497,25.172322,galaxySED/Const.10E10.0005Z.spec.gz,0.095564,0.000686,0.000152,-0.000170,...,2.3,CCM,0.027995,3.1,2562502839,1101,1,2624002907233,53.605763,-28.387497
4,object,179840093598113899,53.231066,-28.457710,21.468666,galaxySED/Exp.25E09.02Z.spec.gz,0.153528,0.000830,0.001565,-0.000518,...,4.0,CCM,0.025146,3.1,2562509140,1102,0,2624009359457,53.231331,-28.457709


In [57]:
host_bulge.to_csv('../../data/%s_bulge_sne_host.csv' % catalog_version)
host_disk.to_csv('../../data/%s_disk_sne_host.csv' % catalog_version)